In [ ]:
import json


def read_accs(dir_toy):
    js = []
    for i in range(1, 10):
        with open(f'{dir_toy}/0.{i}/accs.json') as f:
            j = json.load(f)
            js.append(j)

    with open(f'{dir_toy}/1.0/accs.json') as f:
        j = json.load(f)
        js.append(j)
        
    accs = {}
    for k in js[0]:
        accs[k] = [j[k] for j in js]
    return accs

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer.vocab['good']

In [ ]:
config = {
  'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
#     'height': 500,
#     'width': 800,
    # 'scale': 2 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

In [ ]:
import plotly.graph_objects as go
import plotly.colors
import plotly.express

def plot_accs(accs):
    xs = [i / 10 for i in range(1, 11)]
    figure = go.Figure()
    figure.add_trace(
        go.Scatter(x=xs, y=accs['test-0'], name='D0 L0')
    )
    figure.add_trace(
        go.Scatter(x=xs, y=accs['test-0-shift'], name='D0 L1')
    )
    figure.add_trace(
        go.Scatter(x=xs, y=accs['test-1'], name='D1 L0')
    )
    figure.add_trace(
        go.Scatter(x=xs, y=accs['test-1-shift'], name='D1 L1')
    )
    return figure


def plot_2accs(accs1, accs2, factor='mixing'):
    colors = plotly.colors.qualitative.Plotly

    xs = [i / 10 for i in range(1, 11)]
    figure = go.Figure()
    
    keys = ['test-0', 'test-0-shift', 'test-1', 'test-1-shift']
    names = ['A-D1', 'B-D1', 'A-D2', 'B-D2']

    for color, key, name in zip(colors, keys, names):
        figure.add_trace(
            go.Scatter(
                x=xs, y=accs1[key], name=f'{name} w/  {factor}', 
                line=dict(color=color, width=2, dash='solid')
            )
        )
        figure.add_trace(
            go.Scatter(
                x=xs, y=accs2[key], name=f'{name} w/o {factor}', 
                line=dict(color=color, width=2, dash='dot')
            )
        )
    return figure


def plot_accss(accss, suffixes, mode=None, hide_legend=False, show_legends=None, skips=[]):
    colors = plotly.colors.qualitative.Plotly

    xs = [i / 10 for i in range(1, 11)]
    figure = go.Figure()
    
    if mode is None:
        keys = ['test-0', 'test-0-shift', 'test-1', 'test-1-shift']
    elif mode == 'level-up':
        keys = ['test-0', 'test-0-tr', 'test-1-tr', 'test-1']
    elif mode == 'level-up-shift':
        keys = ['test-0', 'test-0-tr-shift', 'test-1-tr', 'test-1-shift']
        
    names = ['A-D1', 'B-D1', 'A-D2', 'B-D2']
    if len(accss) <= 3:
        dashes = ['solid', 'dash', 'dot', 'longdash', 'dashdot', ]
    else:
        dashes = ['solid', 'longdash', 'dash', 'dashdot', 'dot', ]
    symbols = ['circle-open', 'square-open', 'diamond-open', 'star-open', 'x-open', 'triangle-up',
               'pentagon', 'hexagram', 'star', 'diamond', 'hourglass',
               'bowtie', 'asterisk']

    for i, (color, key, name) in enumerate(zip(colors, keys, names)):
        for j, (accs, suffix, dash, symbol) in enumerate(zip(accss, suffixes, dashes, symbols)):
            if j in skips:
                continue
            figure.add_trace(
                go.Scatter(
                    x=xs, y=accs[key], name=f'{name} {suffix}', 
                    line=dict(color=color, width=3, dash=dash),
                    marker_symbol=symbol,
                    marker=dict(size=10),
                    showlegend=True if show_legends is None else (i in show_legends)
                )
            )
    
    figure.update_layout(
        autosize=False,
        width=800,
        height=470,
        font=dict(
            size=20
        ),
        xaxis_title="Ratio of training data",
        yaxis_title="Accuracy",
        legend=dict(
            orientation='h',
            yanchor="top",
            y=0,
            xanchor="center",
            x=0.5
        ),
        margin=dict(l=0, r=0, t=0, b=0),
        xaxis=dict(side='top'),
        showlegend=True
    )

    if hide_legend:
        figure.update_layout(
            margin=dict(t=0),
            showlegend=False,
            height=350,
        )
    else:
        figure.update_layout(
            height=410,
        )
        
    if show_legends is not None:
        figure.update_layout(
            height=410
        )
    
    return figure

In [ ]:
accs = read_accs('../models/toys/')
accs_shift = read_accs('../models/toys-shift/')
accs_mix = read_accs('../models/toys-mix/')
accs_shuffle = read_accs('../models/toys-mix-shuffle/')
accs_w2v = read_accs('../models/toys-w2v//')
accss = [accs_mix, accs_shift, accs_shuffle, accs_w2v, accs]
names = ['w/ dh', 'w/o dh', 'shuffle weights', 'w2v', 'scratch']
indices = [0, 1, 3, 4]
skips = [2, 3]

fig = plot_accss([accss[i] for i in indices], [names[i] for i in indices], hide_legend=False, skips=skips)

fig.show(config=config)

In [ ]:
accs = read_accs('../models/toys-l0.5/')
accs_shift = read_accs('../models/toys-shift-l0.5/')
accs_mix = read_accs('../models/toys-mix-l0.5/')
accs_shuffle = read_accs('../models/toys-mix-shuffle-l0.5/')
accs_w2v = read_accs('../models/toys-w2v-l0.5/')
accss = [accs_mix, accs_shift, accs_shuffle, accs_w2v, accs]
names = ['w/ dh', 'w/o dh', 'shuffle weights', 'w2v', 'scratch']
indices = [0, 1, 3, 4]
skips = [2, 3]

fig = plot_accss([accss[i] for i in indices], [names[i] for i in indices], hide_legend=True, skips=skips)

fig.show(config=config)

In [ ]:
accs = read_accs('../models/toys-l0.1/')
accs_shift = read_accs('../models/toys-shift-l0.1/')
accs_mix = read_accs('../models/toys-mix-l0.1/')
accs_shuffle = read_accs('../models/toys-mix-shuffle-l0.1/')
accs_w2v = read_accs('../models/toys-w2v-l0.1/')
accss = [accs_mix, accs_shift, accs_shuffle, accs_w2v, accs]
names = ['w/ DH', 'w/o DH', 'shuffle weights', 'CBOW', 'scratch']
indices = [0, 1, 3, 4]
skips = [0, 1]

fig = plot_accss([accss[i] for i in indices], [names[i] for i in indices], hide_legend=False, skips=skips)

fig.show(config=config)

# Level Up

## 100% L1

In [ ]:
lu_accs_mix_shift = read_accs('../models/level-up-mix-shift/')
lu_accs_shift = read_accs('../models/level-up-shift/')
lu_accs_mix_shuffle = read_accs('../models/level-up-mix-shift-shuffle/')
level_up = read_accs('../models/level-up-scratch/')
lu_accs_w2v = read_accs('../models/level-up-w2v//')

accss = [lu_accs_mix_shift, lu_accs_shift, lu_accs_mix_shuffle, lu_accs_w2v, level_up]
names = ['w/ DH', 'w/o DH', 'shuffle weights', 'w2v', 'scratch']
indices = [0, 1, 3, 4]

fig = plot_accss(
    [accss[i] for i in indices], [names[i] for i in indices],
    mode='level-up-shift',
    hide_legend=True
)

fig.update_layout(yaxis_range=[0.45,0.92])
fig.show(config=config)



In [ ]:
lu_accs_mix = read_accs('../models/level-up-mix/')
lu_accs = read_accs('../models/level-up-shared/')
lu_accs_mix_shuffle = read_accs('../models/level-up-mix-shuffle/')
level_up = read_accs('../models/level-up-scratch-shared/')
lu_accs_w2v = read_accs('../models/level-up-w2v//')


# figure = plot_accss(
#     [lu_accs_mix, lu_accs_shift, lu_accs_mix_shuffle, lu_accs_w2v, level_up], 
#     ['w/mix', 'w/o mix', 'mix shuffle', 'w2v', 'scratch'], 
#     mode='level-up'
# )
# figure.show()


accss = [lu_accs_mix, lu_accs, lu_accs_mix_shuffle, lu_accs_w2v, level_up]
names = ['w/ DH', 'w/o DH', 'shuffle weights', 'w2v', 'scratch']
indices = [0, 1, 3, 4]

fig = plot_accss(
    [accss[i] for i in indices], [names[i] for i in indices],
    mode='level-up',
    hide_legend=True
)
fig.update_layout(yaxis_range=[0.45, 0.92])

fig.show(config=config)



## 90% L1

In [ ]:
lu_accs_mix_shift = read_accs('../models/level-up-mix-shift-l0.1/')
lu_accs_shift = read_accs('../models/level-up-shift-l0.1/')
lu_accs_mix_shuffle = read_accs('../models/level-up-mix-shift-shuffle-l0.1/')
level_up = read_accs('../models/level-up-scratch-l0.1/')
lu_accs_w2v = read_accs('../models/level-up-w2v-l0.1//')

accss = [lu_accs_mix_shift, lu_accs_shift, lu_accs_mix_shuffle, level_up, lu_accs_w2v]
names = ['w/ DH', 'w/o DH', 'shuffle weights', 'CBOW', 'scratch']
indices = [0, 1, 3, 4]

fig = plot_accss(
    [accss[i] for i in indices], [names[i] for i in indices],
    mode='level-up-shift',
    hide_legend=False,
    show_legends=[0, 1]
)
fig.update_layout(yaxis_range=[0.45,0.92])

fig.show(config=config)


In [ ]:
lu_accs_mix = read_accs('../models/level-up-mix-l0.1/')
lu_accs = read_accs('../models/level-up-shared-l0.1/')
lu_accs_mix_shuffle = read_accs('../models/level-up-mix-shuffle-l0.1/')
level_up = read_accs('../models/level-up-scratch-l0.1-shared/')
lu_accs_w2v = read_accs('../models/level-up-mix-w2v-l0.1/')

accss = [lu_accs_mix, lu_accs, lu_accs_mix_shuffle, lu_accs_w2v, level_up]
names = ['w/ DH', 'w/o DH', 'shuffle weights', 'CBOW', 'scratch']
indices = [0, 1, 3, 4]

fig = plot_accss(
    [accss[i] for i in indices], [names[i] for i in indices], 
    mode='level-up',
    hide_legend=False,
    show_legends=[2, 3]
)
fig.update_layout(yaxis_range=[0.45, 0.92])

fig.show(config=config)


## 50% L1

In [ ]:
lu_accs_mix_shift = read_accs('../models/level-up-mix-shift-l0.5/')
lu_accs_shift = read_accs('../models/level-up-shift-l0.5/')
lu_accs_mix_shuffle = read_accs('../models/level-up-mix-shift-shuffle-l0.5/')
level_up = read_accs('../models/level-up-scratch-l0.5/')
lu_accs_w2v = read_accs('../models/level-up-w2v-l0.5//')


accss = [lu_accs_mix_shift, lu_accs_shift, lu_accs_mix_shuffle, lu_accs_w2v, level_up, ]
names = ['w/ DH', 'w/o DH', 'shuffle weights', 'w2v', 'scratch']
indices = [0, 1, 3, 4]

fig = plot_accss(
    [accss[i] for i in indices], [names[i] for i in indices],
    mode='level-up-shift',
    hide_legend=True
)
# fig.update_layout(yaxis_range=[0.45,0.95])
fig.show(config=config)

In [ ]:
lu_accs_mix = read_accs('../models/level-up-mix-l0.5/')
lu_accs = read_accs('../models/level-up-shared-l0.5/')
lu_accs_mix_shuffle = read_accs('../models/level-up-mix-shuffle-l0.5/')
level_up = read_accs('../models/level-up-scratch-l0.5-shared/')
lu_accs_w2v = read_accs('../models/level-up-mix-w2v-l0.5//')


accss = [lu_accs_mix, lu_accs, lu_accs_mix_shuffle, lu_accs_w2v, level_up]
names = ['w/ DH', 'w/o DH', 'shuffle weights', 'CBOW', 'scratch']
indices = [0, 1, 3, 4]

fig = plot_accss(
    [accss[i] for i in indices], [names[i] for i in indices],
    mode='level-up',
    hide_legend=False
)
# fig.update_layout(yaxis_range=[0.45,0.95])
fig.show(config=config)
